In [1]:
import tensorflow as tf
from train import build_model
from preprocess_data import load_data
from layers import *

2024-07-09 11:44:46.359830: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-09 11:44:46.439523: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-09 11:44:46.442050: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-09 11:44:48.028912: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
batch_size = 32
train_dataset, test_dataset = load_data(data_path="/home/pbr-student/personal/thesis/test/PedestrianTrajectoryPrediction/final_dataset.pkl", batch_size=batch_size)

2024-07-09 11:44:54.922640: W tensorflow/core/framework/dataset.cc:956] Input of Window will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.
2024-07-09 11:45:24.930469: W tensorflow/core/framework/dataset.cc:956] Input of Window will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


In [3]:
len(train_dataset)

69

In [3]:
model = build_model()
for (batch_x1, batch_x2) in train_dataset.take(1):
        output2 = model.predict((batch_x1, batch_x2))
        break

batch_size: 32
sequence_length: 15
mask aaray: (32, 15)
batch_size: 32
sequence_length: 15
mask aaray: (32, 15)
1/1 [==============================] - 1s 1s/step


In [4]:
print(output2.keys())

dict_keys(['mask', 'position', 'position_raw_scale', 'mixture_logits', 'targets'])


In [5]:
output2['position_raw_scale'].shape

(32, 15, 5, 6)

In [4]:
import tensorflow_models as tfm
import tensorflow_probability as tfp

In [5]:
# 2) Create cosine learning rate schedule w/ warmup.
def _get_learning_rate_schedule(
    warmup_steps: int,
    total_steps: int,
    learning_rate: float,
    alpha: float = 0.0) -> tf.keras.optimizers.schedules.LearningRateSchedule:
  """Returns a cosine decay learning rate schedule to be used in training.

  Args:
    warmup_steps: Number of training steps to apply warmup. If global_step <
      warmup_steps, the learning rate will be `global_step/num_warmup_steps *
      init_lr`.
    total_steps: The total number of training steps.
    learning_rate: The peak learning rate anytime during the training.
    alpha: The alpha parameter forwarded to CosineDecay

  Returns:
    A CosineDecay learning schedule w/ warmup.
  """

  decay_schedule = tf.keras.optimizers.schedules.CosineDecay(
      initial_learning_rate=learning_rate, decay_steps=total_steps, alpha=alpha)
  return tfm.optimization.LinearWarmup(decay_schedule, warmup_steps, 1e-10)

learning_rate_schedule = _get_learning_rate_schedule(
    warmup_steps=5e4, total_steps=5e6,
    learning_rate=1e-4)

In [7]:
optimizer = tf.keras.optimizers.Adam(
    learning_rate=learning_rate_schedule,
    global_clipnorm=1.)

In [26]:
class Loss(object):
  """Base class for Human Scene Transformer Losses.
  """

  def __init__(self, name='Loss', clip_loss_max=tf.float32.max):
    self.clip_loss_max = clip_loss_max
    self.name = name

  @tf.function
  def __call__(self, input_batch, predictions):
    return self.call(input_batch, predictions)

  def call(self, input_batch, predictions):
    """Calculates loss for fields which should be predicted."""

    # [b, a, t]
    ##should_predict = input_batch['should_predict'][..., 0]
    should_predict = tf.math.logical_not(predictions['mask'])
    # [b, a, t]
    loss_per_batch = self.get_per_batch_loss(input_batch, predictions)

    loss_per_batch = tfp.math.clip_by_value_preserve_gradient(
        loss_per_batch,
        tf.float32.min,
        self.clip_loss_max)

    # Compute loss only on positions w/ should_predict == True.
    print("should predict Loss:", should_predict.shape)
    should_predict_ind = tf.where(should_predict)
    loss_should_predict_mat = tf.gather_nd(
        params=loss_per_batch, indices=should_predict_ind)

    loss_should_predict = tf.reduce_mean(loss_should_predict_mat)
    # If there are no agents to be predicted xyz_loss_should_predict can be NaN
    loss_should_predict = tf.math.multiply_no_nan(
        loss_should_predict,
        tf.cast(tf.math.reduce_any(should_predict), tf.float32))

    loss_dict = {
        'loss': loss_should_predict,
        f'{self.name}_loss': loss_should_predict
    }
    return loss_dict

  def get_per_batch_loss(self, input_batch, predictions):
    raise NotImplementedError

class MinNLLPositionMixtureCategoricalCrossentropyLoss(Loss):
  """MinNLLPositionNLLLoss and MixtureCategoricalCrossentropyLoss."""

  def __init__(self, **kwargs):
    super().__init__(name='MinNLLMixture', **kwargs)
    self.position_loss_obj = MinNLLPositionLoss()
    self.mixture_loss_obj = MinNLLMixtureCategoricalCrossentropyLoss()

  def call(self, input_batch, predictions):
    position_loss = self.position_loss_obj(input_batch, predictions)
    print("position loss: ", position_loss)
    mixture_loss = self.mixture_loss_obj(input_batch, predictions)
    print("mixture loss: ", mixture_loss)

    loss = position_loss['loss'] + mixture_loss['loss']

    print("loss: ", loss)

    loss_dict = {**position_loss, **mixture_loss}

    loss_dict['loss'] = loss

    return loss_dict
  
class PositionNLLLoss(Loss):
  """Position NLL Loss for human trajectory predictions."""

  def __init__(self, **kwargs):
    super().__init__(name='position', **kwargs)

  # from model.output_distributions
  def force_positive(self, x, eps=1e-6):
    return tf.keras.activations.elu(x) + 1. + eps

  # from model.output_distributions
  def to_positive_definite_scale_tril(self, logit_sigma):
    tril = tfp.math.fill_triangular(logit_sigma)
    scale_tril = tf.linalg.set_diag(
        tril,
        self.force_positive(tf.linalg.diag_part(tril)))
    return scale_tril
  
  # from model.output_distributions
  def get_position_distribution(self, model_output):
    """Multivariate Normal distribution over position."""
    p_pos = tfp.distributions.MultivariateNormalTriL(
        loc=model_output['position'],
        scale_tril=self.to_positive_definite_scale_tril(
            model_output['position_raw_scale']))
    return p_pos

  def get_per_batch_loss(self, input_batch, predictions):
    """Negative log probability of ground truth with respect to predictions."""
    # [b, a, t, 3]
    p_position = self.get_position_distribution(predictions)

    # [b, a, t, 1]
    position_nll = -p_position.log_prob(
        predictions['targets'])
    return position_nll
  
class MinNLLPositionLoss(PositionNLLLoss):
  """MinNLLPositionNLL loss."""

  def __init__(self, **kwargs):
    super().__init__(**kwargs)
    self.mixture_loss = tf.keras.losses.CategoricalCrossentropy(
        from_logits=True, reduction=tf.keras.losses.Reduction.NONE)

  def get_per_batch_loss(self, input_batch, predictions):
    """Negative log probability of mode with smallest ADE."""
    predictions = predictions.copy()

    predictions['targets'] = predictions['targets'][..., tf.newaxis, :]
    # [b, a, t, n]
    position_nll = super().get_per_batch_loss(input_batch, predictions)

    # [b, a, t, 1]
    ##should_predict = tf.cast(input_batch['should_predict'], tf.float32)
    should_predict = tf.math.logical_not(predictions['mask'])
    should_predict = tf.expand_dims(should_predict, axis=-1)
    should_predict = tf.cast(should_predict, tf.float32)

    # [b, a, t, n, 1]
    per_position_nll = (
        position_nll[..., tf.newaxis] * should_predict[..., tf.newaxis, :]
    )

    # Get mode with minimum NLL
    # [b, a, n, 1]
    per_mode_nll_sum = tf.reduce_sum(per_position_nll, axis=2)

    t = tf.shape(position_nll)[2]

    # [b, a, 1]
    min_nll_indices = tf.math.argmin(per_mode_nll_sum, axis=-2)

    # [b, a, t, 1]
    min_nll_indices_tiled = tf.tile(
        min_nll_indices[..., tf.newaxis], [1, t, 1])

    # [b, a, t]
    position_nll_min_ade = tf.gather(
        position_nll, indices=min_nll_indices_tiled, batch_dims=2, axis=-1
        )[..., 0]

    return position_nll_min_ade

class MinNLLMixtureCategoricalCrossentropyLoss(PositionNLLLoss):
  """Categorical Corssentropy Loss for Mixture Weight."""

  def __init__(self, **kwargs):
    super().__init__(**kwargs)
    self.mixture_loss = tf.keras.losses.CategoricalCrossentropy(
        from_logits=True, reduction=tf.keras.losses.Reduction.NONE)

  def get_per_batch_loss(self, input_batch, predictions):
    """Negative log probability of ground truth with respect to predictions."""
    predictions= predictions.copy()
    predictions['targets'] = predictions[
        'targets'][..., tf.newaxis, :]

    # Calculate ADE
    # [b, a, t, n, 1]
    position_nll = super().get_per_batch_loss(input_batch, predictions)
    position_nll = tf.expand_dims(position_nll, axis=-1)

    # [b, a, t, 1]
    ##should_predict = tf.cast(input_batch['should_predict'], tf.float32)
    should_predict = tf.cast(tf.expand_dims(tf.math.logical_not(predictions['mask']), axis=-1), tf.float32)

    # [b, a, t, n, 1]
    per_position_nll = (
        position_nll * should_predict[..., tf.newaxis, :]
    )

    # Get mode with minimum NLL
    # [b, a, n, 1]
    ##per_mode_nll_sum = tf.reduce_sum(per_position_nll, axis=2)
    per_mode_nll_sum = tf.reduce_sum(per_position_nll, axis=1)

    a = tf.shape(position_nll)[1]
    #n = tf.shape(position_nll)[3]
    n = tf.shape(position_nll)[2]

    # [b, a, 1]
    min_nll_indices = tf.math.argmin(per_mode_nll_sum, axis=-2)

    # [b, a, n]
    min_nll_indices_one_hot = tf.one_hot(min_nll_indices[..., 0], n)

    # [b, a]
    # TODO
    #mixture_loss = self.mixture_loss(
        #min_nll_indices_one_hot,
        #tf.tile(predictions['mixture_logits'][..., 0, :], [1, a, 1]))
    
    mixture_loss = self.mixture_loss(
        min_nll_indices_one_hot,
        predictions['mixture_logits'][..., 0, :])

    return mixture_loss

  def call(self, input_batch, predictions):
    """Calculates loss."""

    # [b, a]
    ##should_predict = tf.reduce_any(
      ##  input_batch['should_predict'][..., 0], axis=-1)
    should_predict = tf.reduce_any(
        tf.math.logical_not(predictions['mask'][..., 0]), axis=-1)
    # [b, a]
    loss_per_batch = self.get_per_batch_loss(input_batch, predictions)

    # Compute loss only on positions w/ should_predict == True.
    print("predictions: ", predictions['mask'].shape)
    print("shuld_predict: ", should_predict.shape)
    should_predict_ind = tf.where(should_predict)
    loss_should_predict_mat = tf.gather_nd(
        params=loss_per_batch, indices=should_predict_ind)

    loss_should_predict = tf.reduce_mean(loss_should_predict_mat)
    # If there are no agents to be predicted xyz_loss_should_predict can be NaN
    loss_should_predict = tf.math.multiply_no_nan(
        loss_should_predict,
        tf.cast(tf.math.reduce_any(should_predict), tf.float32))

    # Mixture weights are per scene. So we do not have to mask anything
    loss_dict = {
        'loss': loss_should_predict,
        f'{self.name}_loss': loss_should_predict
    }
    return loss_dict
  
loss_obj = MinNLLPositionMixtureCategoricalCrossentropyLoss()
#loss = loss_obj(input_batch=batch_x1, predictions=output2)

In [27]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 15, 3)]              0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 15, 51)]             0         []                            
                                                                                                  
 preprocess_layer (Preproce  (((32, 15, 3),               0         ['input_1[0][0]',             
 ssLayer)                     (32, 15, 51)),                         'input_2[0][0]']             
                              (32, 15),                                                           
                              (32, 15, 3))                                                    

In [27]:
# Create metrics.

class Mean(tf.keras.metrics.Mean):

  def __init__(self, name="mean", dtype=None):
    super().__init__(name=name, dtype=dtype)

def distance_error(target: tf.Tensor, pred: tf.Tensor) -> tf.Tensor:
  return tf.sqrt(
      tf.reduce_sum(tf.square(pred - target), axis=-1, keepdims=True))

class ADE(tf.keras.metrics.Metric):
  """Average Displacement Error over a n dimensional track.

  Calculates the mean L2 distance over all predicted timesteps.
  """
  # TODO: timestep?
  def __init__(self, num_history_steps=6, timestep=0.4, cutoff_seconds=None, at_cutoff=False, name='ADE'):
    """Initializes the ADE metric.

    Args:
      params: ModelParams
      cutoff_seconds: Cutoff up to which time the metric should be calculated
        in seconds.
      at_cutoff: If True metric will be calculated at cutoff timestep.
        Otherwise metric is calculated as average up to cutoff_seconds.
      name: Metric name.
    """
    super().__init__(name=name)
    self.cutoff_seconds = cutoff_seconds
    self.at_cutoff = at_cutoff
    if cutoff_seconds is None:
      self.cutoff_idx = None
    else:
      # +1 due to current time step.
      self.cutoff_idx = int(
          num_history_steps +
          cutoff_seconds / timestep) + 1

    self.num_predictions = self.add_weight(
        name='num_predictions', initializer='zeros')
    self.total_deviation = self.add_weight(
        name='total_deviation', initializer='zeros')

  def _reduce(self, ade_with_modes, input_batch, predictions):
    """Reduces mode dimension. The base class squeezes a single mode."""
    return tf.squeeze(ade_with_modes, axis=-1)

  def update_state(self, input_batch, predictions):
    should_predict = tf.cast(tf.math.logical_not(predictions['mask']), tf.float32)
    should_predict = tf.expand_dims(should_predict, axis=-1)

    target = predictions['targets']
    target = target[..., :predictions['position'].shape[-1]]
    # [b, a, t, n, 3] -> [b, a, t, n, 1]
    per_position_ade = distance_error(
        target[..., tf.newaxis, :],
        predictions['position'])

    # Non-observed or past should not contribute to ade.
    deviation = tf.math.multiply_no_nan(per_position_ade,
                                        should_predict[..., tf.newaxis, :])
    # Chop off the un-wanted time part.
    # [b, a, cutoff_idx, 1]
    if self.at_cutoff and self.cutoff_seconds is not None:
      deviation = deviation[:, self.cutoff_idx-1:self.cutoff_idx, :]
      num_predictions = tf.reduce_sum(
          should_predict[:, self.cutoff_idx-1:self.cutoff_idx, :])
    else:
      deviation = deviation[:, :self.cutoff_idx, :]
      num_predictions = tf.reduce_sum(should_predict[:, :self.cutoff_idx, :])
    
    # Reduce along time
    deviation = tf.reduce_sum(deviation, axis=1)
    # Reduce along modes
    deviation = self._reduce(deviation, input_batch, predictions)
    # [1]
    deviation = tf.reduce_sum(deviation)

    self.num_predictions.assign_add(num_predictions)
    self.total_deviation.assign_add(deviation)

  def result(self):
    return self.total_deviation / self.num_predictions

  def reset_states(self):
    self.num_predictions.assign(0)
    self.total_deviation.assign(0.0)

class MinADE(ADE):
  """Takes the minimum over all modes."""

  def _reduce(self, ade_with_modes, input_batch, predictions):
    return tf.reduce_min(ade_with_modes, axis=-2)

class MLADE(ADE):
  """Takes the maximum likelihood mode."""

  def _reduce(self, ade_with_modes, input_batch, predictions):
    # Get index of mixture component with highest probability
    # [b, a=1, t=1, n]
    ml_indices = tf.math.argmax(predictions['mixture_logits'], axis=-1)
    a = ade_with_modes.shape[1]
    ml_indices = tf.tile(
        tf.squeeze(ml_indices, axis=1), [a])[..., tf.newaxis]

    return tf.gather(
        ade_with_modes, indices=ml_indices, batch_dims=2, axis=-2)[..., 0, :]

def force_positive(x, eps=1e-6):
  return tf.keras.activations.elu(x) + 1. + eps

def to_positive_definite_scale_tril(logit_sigma):
  tril = tfp.math.fill_triangular(logit_sigma)
  scale_tril = tf.linalg.set_diag(
      tril,
      force_positive(tf.linalg.diag_part(tril)))
  return scale_tril

def get_position_distribution(model_output):
  """Multivariate Normal distribution over position."""
  p_pos = tfp.distributions.MultivariateNormalTriL(
      loc=model_output['position'],
      scale_tril=to_positive_definite_scale_tril(
          model_output['position_raw_scale']))

  return p_pos

def get_multimodal_position_distribution(model_output):
  """Multivariate Normal Mixture distribution over position."""
  p_pos = get_position_distribution(model_output)

  p_pos_mm = tfp.distributions.MixtureSameFamily(
      mixture_distribution=tfp.distributions.Categorical(
          logits=model_output['mixture_logits']),
      components_distribution=p_pos)

  return p_pos_mm

class PositionNegativeLogLikelihood(tf.keras.metrics.Metric):
  """Position Negative Log Likelihood."""

  def __init__(self, num_history_steps=6, timestep=0.4,cutoff_seconds=None, at_cutoff=False,
               name='PosNLL'):
    """Initializes the PositionNegativeLogLikelihood metric.

    Args:
      params: ModelParams
      cutoff_seconds: Cutoff up to which time the metric should be calculated
        in seconds.
      at_cutoff: If True metric will be calculated at cutoff timestep.
        Otherwise metric is calculated as average up to cutoff_seconds.
      name: Metric name.
    """
    super().__init__(name=name)
    self.cutoff_seconds = cutoff_seconds
    self.at_cutoff = at_cutoff
    if cutoff_seconds is None:
      self.cutoff_idx = None
    else:
      # +1 due to current time step.
      self.cutoff_idx = int(
          num_history_steps +
          cutoff_seconds / timestep) + 1

    self.num_predictions = self.add_weight(
        name='num_predictions', initializer='zeros')
    self.total_deviation = self.add_weight(
        name='total_deviation', initializer='zeros')

  def update_state(self, input_batch, predictions):
    should_predict = tf.cast(tf.math.logical_not(predictions['mask']), tf.float32)
    should_predict = tf.expand_dims(should_predict, axis=-1)

    p_pos = get_multimodal_position_distribution(predictions)

    target = predictions['targets']
    target = target[..., :p_pos.event_shape_tensor()[0]]
    print(target.shape)

    # [b, a, t, n, 1]
    per_position_nll = -p_pos.log_prob(target)[..., tf.newaxis]

    # Non-observed or past should not contribute to metric.
    nll = tf.math.multiply_no_nan(per_position_nll, should_predict)
    # Chop off the un-wanted time part.
    # [b, a, cutoff_idx, 1]
    if self.at_cutoff and self.cutoff_seconds is not None:
      nll = nll[:, self.cutoff_idx-1:self.cutoff_idx, :]
      num_predictions = tf.reduce_sum(
          should_predict[:, self.cutoff_idx-1::self.cutoff_idx, :])
    else:
      nll = nll[:, :self.cutoff_idx, :]
      num_predictions = tf.reduce_sum(should_predict[:, :self.cutoff_idx, :])

    # [1]
    nll = tf.reduce_sum(nll)

    self.num_predictions.assign_add(num_predictions)
    self.total_deviation.assign_add(nll)

  def result(self):
    return self.total_deviation / self.num_predictions

  def reset_states(self):
    self.num_predictions.assign(0)
    self.total_deviation.assign(0.0)


In [28]:
import os
import datetime

model_base_dir = ""
dt_str = datetime.datetime.now().strftime('%Y-%m-%d_%H:%M:%S')
model_dir = os.path.join(model_base_dir, dt_str)
os.makedirs(model_dir)
ckpt_dir = os.path.join(model_dir, 'ckpts')
os.makedirs(ckpt_dir)
ckpt_best_dir = os.path.join(model_dir, 'ckpts_best')
os.makedirs(ckpt_best_dir)
checkpoint_prefix = os.path.join(ckpt_dir, 'ckpt')
checkpoint_prefix_best = os.path.join(ckpt_best_dir, 'ckpt')
tensorboard_dir = '/tmp/tensorboard'


train_summary_writer = tf.summary.create_file_writer(
      os.path.join(tensorboard_dir, 'train'))
eval_summary_writer = tf.summary.create_file_writer(
    os.path.join(tensorboard_dir, 'eval'))


batches_per_train_step=1 #25000
batches_per_eval_step =1 # 2000
eval_every_n_step = 1 #1e4

strategy = tf.distribute.OneDeviceStrategy('cpu')

dist_train_dataset = strategy.experimental_distribute_dataset(train_dataset)
dist_eval_dataset = strategy.experimental_distribute_dataset(test_dataset)

current_global_step = 0

with strategy.scope():
    model = build_model()
    optimizer = tf.keras.optimizers.Adam(
        learning_rate=learning_rate_schedule,
        global_clipnorm=1.)
    loss_obj = MinNLLPositionMixtureCategoricalCrossentropyLoss()
    train_metrics = {
    'loss': Mean(),
    'loss_position': Mean(),
    'min_ade': MinADE(),
    'ml_ade': MLADE(),
    'pos_nll': PositionNegativeLogLikelihood()
    }

    eval_metrics = {
      'loss': Mean(),
      'loss_position': Mean(),
      'min_ade': MinADE(),
      'ml_ade': MLADE(),
      'pos_nll': PositionNegativeLogLikelihood()
    }

best_eval_loss = tf.Variable(tf.float32.max)
checkpoint = tf.train.Checkpoint(model=model,
                                   optimizer=optimizer,
                                   best_eval_loss=best_eval_loss)
checkpoint_best = tf.train.Checkpoint(model=model)
best_checkpoint_manager = tf.train.CheckpointManager(checkpoint_best,
                                                    checkpoint_prefix_best,
                                                    max_to_keep=1)

2024-07-09 13:33:45.167485: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:786] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_1"
op: "TensorSliceDataset"
input: "Placeholder/_0"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_DOUBLE
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 2783
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\030TensorSliceDataset:14786"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 15
        }
        dim {
          size: 3
        }
      }
    }
  }
}
attr {
  key: "replicate_on_split"
  value {
    b: false
  }
}
experimental_type {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_DATASET
    args {
      type_id: TFT_PRODUCT
      args {
        type_id: TFT_TENSO

batch_size: 32
sequence_length: 15
mask aaray: (32, 15)


In [29]:
# Define Training and Eval tf.function.
@tf.function
def train_step(iterator):
    """Training function."""

    def step_fn(input_batch):
        print("input batch ", input_batch[0].shape)
        with tf.GradientTape() as tape:
            print("predict")
            predictions = model(input_batch, training=True)
            print("loss")
            loss_dict = loss_obj(input_batch, predictions)
            loss = (loss_dict['loss']
                / tf.cast(strategy.num_replicas_in_sync, tf.float32))
        print("grad")
        grads = tape.gradient(loss, model.trainable_variables)
        print("optimizer")
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        # Update the training metrics.
        # These need special treatments as they are standard keras metrics.
        print("train metrics")
        train_metrics['loss'].update_state(loss_dict['loss'])
        print("train metrics2")
        train_metrics['loss_position'].update_state(loss_dict['position_loss'])
        # Our own metrics.
        for key in train_metrics:
            if key in {'loss', 'loss_position', 'loss_orientation'}:
                continue
            print("key ", key)
            train_metrics[key].update_state(input_batch, predictions)
        print("fertig")

    for _ in tf.range(tf.constant(batches_per_train_step)):
        strategy.run(
            step_fn,
            args=(next(iterator),),
            options=tf.distribute.RunOptions(
                experimental_enable_dynamic_batch_size=False))

@tf.function
def eval_step(iterator):

    def step_fn(input_batch):
        print("eval step")
        predictions = model(input_batch, training=False)
        loss_dict = loss_obj(input_batch, predictions)
        # Update the eval metrics.
        # These need special treatments as they are standard keras metrics.
        eval_metrics['loss'].update_state(loss_dict['loss'])
        eval_metrics['loss_position'].update_state(loss_dict['position_loss'])
        # Our own metrics.
        for key in eval_metrics:
            if key in {'loss', 'loss_position', 'loss_orientation'}:
                continue
            eval_metrics[key].update_state(input_batch, predictions)

    for _ in tf.range(tf.constant(batches_per_eval_step)):
        strategy.run(
            step_fn,
            args=(next(iterator),),
            options=tf.distribute.RunOptions(
                experimental_enable_dynamic_batch_size=False))

In [11]:
from absl import logging

In [30]:
 # 5) Actual Training Loop
train_iter = iter(dist_train_dataset)
eval_iter = iter(dist_eval_dataset)
total_train_steps = 2 # 1e6
num_train_iter = (
    total_train_steps // batches_per_train_step)
current_train_iter = (
    current_global_step // batches_per_train_step)

print("range: ",range(current_train_iter, num_train_iter))

logging.info('Beginning training.')
for step in range(current_train_iter, num_train_iter):
    print("step: ", step)
    # Actual number of SGD steps.
    actual_steps = step * batches_per_train_step
    with train_summary_writer.as_default():
        # Run training SGD over train_param.batches_per_train_step batches.
        # optimizer.iterations = step * train_param.batches_per_train_step.
        print("train step")
        train_step(train_iter)
        print("out")
        # Writing metrics to tensorboard.
        if step % 1 == 0:
            for key in train_metrics:
                print(train_metrics[key].result())
                tf.summary.scalar(
                    key, train_metrics[key].result(), step=optimizer.iterations)

            if isinstance(optimizer, tf.keras.optimizers.experimental.Optimizer):
                learning_rate = optimizer.learning_rate
            else:
                learning_rate = optimizer.lr(optimizer.iterations)
            tf.summary.scalar(
                'learning_rate',
                learning_rate,
                step=optimizer.iterations)
            logging.info('Training step %d', step)
            print('Training loss: %.4f, ADE: %.4f',
                            train_metrics['loss'].result().numpy(),
                            train_metrics['min_ade'].result().numpy())
            # Reset metrics.
            for key in train_metrics:
                train_metrics[key].reset_states()

    # Evaluation.
    if actual_steps % eval_every_n_step == 0:
        print("eval")
        logging.info('Evaluating step %d over %d random eval samples', step,
                    batches_per_eval_step * batch_size)
        with eval_summary_writer.as_default():
            eval_step(eval_iter)
            for key in eval_metrics:
                tf.summary.scalar(
                    key, eval_metrics[key].result(), step=optimizer.iterations)
            logging.info('Eval loss: %.4f, ADE: %.4f',
                            eval_metrics['loss'].result().numpy(),
                            eval_metrics['min_ade'].result().numpy())

            if eval_metrics['loss'].result() < best_eval_loss:
                best_eval_loss.assign(eval_metrics['loss'].result())
                best_checkpoint_manager.save()

            # Reset metrics.
            for key in eval_metrics:
                eval_metrics[key].reset_states()

            # Save model.
            checkpoint_name = checkpoint.save(checkpoint_prefix)
            logging.info('Saved checkpoint to %s', checkpoint_name)

2024-07-09 13:33:53.450222: W tensorflow/core/framework/dataset.cc:956] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


range:  range(0, 2)
step:  0
train step
input batch  (None, 15, 3)
predict
batch_size: 32
sequence_length: 15
mask aaray: (32, 15)
loss
position loss:  {'loss': <tf.Tensor 'PartitionedCall:0' shape=() dtype=float32>, 'position_loss': <tf.Tensor 'PartitionedCall:1' shape=() dtype=float32>}
predictions:  (32, 15)
shuld_predict:  ()
mixture loss:  {'loss': <tf.Tensor 'PartitionedCall_1:0' shape=() dtype=float32>, 'position_loss': <tf.Tensor 'PartitionedCall_1:1' shape=() dtype=float32>}
loss:  Tensor("add:0", shape=(), dtype=float32)
grad
optimizer
train metrics
train metrics2
key  min_ade
key  ml_ade
key  pos_nll
(32, 15, 3)
fertig
input batch  (None, 15, 3)
predict
batch_size: 32
sequence_length: 15
mask aaray: (32, 15)
loss
grad
optimizer
train metrics
train metrics2
key  min_ade
key  ml_ade
key  pos_nll
(32, 15, 3)
fertig


InvalidArgumentError: Graph execution error:

Detected at node 'Where' defined at (most recent call last):
    File "/home/pbr-student/miniconda3/envs/openmmlab/lib/python3.8/runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/pbr-student/miniconda3/envs/openmmlab/lib/python3.8/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/pbr-student/.local/lib/python3.8/site-packages/ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "/home/pbr-student/.local/lib/python3.8/site-packages/traitlets/config/application.py", line 1075, in launch_instance
      app.start()
    File "/home/pbr-student/.local/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 739, in start
      self.io_loop.start()
    File "/home/pbr-student/.local/lib/python3.8/site-packages/tornado/platform/asyncio.py", line 205, in start
      self.asyncio_loop.run_forever()
    File "/home/pbr-student/mini

In [20]:
train_iter = iter(dist_train_dataset)
for step in range(current_train_iter, num_train_iter):
    print("step: ", step)
    print(next(train_iter))

step:  0
(<tf.Tensor: shape=(32, 15, 3), dtype=float64, numpy=
array([[[ 0.04211119, -0.36976998,  3.151     ],
        [-0.0188095 , -0.35409565,  3.103     ],
        [-0.05425067, -0.34582817,  3.119     ],
        ...,
        [-0.42017372, -0.20322584,  2.262     ],
        [-0.45865566, -0.18066558,  2.167     ],
        [-0.47094274, -0.15554524,  2.159     ]],

       [[ 1.70223823, -0.63842558,  4.508     ],
        [ 1.55547465, -0.51670998,  4.173     ],
        [ 1.57094049, -0.53107336,  4.289     ],
        ...,
        [ 1.11928094, -0.51852724,  3.32      ],
        [ 1.11762116, -0.48925476,  3.268     ],
        [ 1.14407304, -0.50511799,  3.268     ]],

       [[ 1.80311369, -0.67323804,  8.235     ],
        [ 1.78978252, -0.68656256,  8.235     ],
        [ 1.78978252, -0.65991351,  8.235     ],
        ...,
        [ 1.45805128, -0.69422637,  6.862     ],
        [ 1.44694279, -0.68312341,  6.862     ],
        [ 1.40664234, -0.66468482,  6.57      ]],

       ...